In [ ]:
#!pip install aquacrop==2.2
#!pip install numba==0.55
#!pip install statsmodels==0.13.2
#!pip install SALib
#!pip install -U scikit-learn scipy matplotlib
!pip install --upgrade pyswarm  #https://pythonhosted.org/pyswarm/

In [2]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse
from aquacrop.utils import prepare_weather, get_filepath
from aquacrop import AquaCropModel, Soil, Crop, InitialWaterContent, IrrigationManagement
#from aquacrop.entities import IrrigationManagement

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import pickle 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import sklearn.metrics as metrics
from pyswarm import pso
import os
from os import chdir, getcwd


In [3]:
os.chdir('/home/jupyter-wndlovu/') # change working directory

wd=getcwd()

In [ ]:
def soilCompart(soilList):
    custom_soil = []
    for df in soilList:

        #soils_df = df.groupby(['Year'])
        soils_df = [v for k, v in df.groupby('Year')]
        #print(soils_df)
        for j in soils_df:
            #df_list = []
            soils_df_pivot = j.drop(['crop_mn_codeyear', 'Year'], axis=1)
            #print(soils_df_pivot)
            for i, row in soils_df_pivot.iterrows():
                #print(row)
                #print(soils_df_pivot)
                if row['depth'] == "0_5":
                    pred_thWP_5 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                    wp_5 = pred_thWP_5 + (0.14 * pred_thWP_5) - 0.02
                    pred_thFC_5 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                    fc_5 = pred_thFC_5 + (1.283 * (np.power(pred_thFC_5, 2))) - (0.374 * pred_thFC_5) - 0.015
                        #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                    ts_5 =soils_df_pivot["theta_s"][i]
                    ks_5=(soils_df_pivot['ksat'][i])*240
                if row['depth'] == "100_200":
                    pred_thWP_200 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                    wp_200 = pred_thWP_200 + (0.14 * pred_thWP_200) - 0.02
                    pred_thFC_200 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                    fc_200 = pred_thFC_200 + (1.283 * (np.power(pred_thFC_200, 2))) - (0.374 * pred_thFC_200) - 0.015
                    #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                    ts_200 =soils_df_pivot["theta_s"][i]
                    ks_200=(soils_df_pivot['ksat'][i])*240
                if row['depth'] == "15_30":
                    pred_thWP_30 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                    wp_30 = pred_thWP_30 + (0.14 * pred_thWP_30) - 0.02
                    pred_thFC_30 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                    fc_30 = pred_thFC_30 + (1.283 * (np.power(pred_thFC_30, 2))) - (0.374 * pred_thFC_30) - 0.015
                    #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                    ts_30 =soils_df_pivot["theta_s"][i]
                    ks_30=(soils_df_pivot['ksat'][i])*240
                if row['depth'] == "30_60":
                    pred_thWP_60 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                    wp_60 = pred_thWP_60 + (0.14 * pred_thWP_60) - 0.02
                    pred_thFC_60 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                    fc_60 = pred_thFC_60 + (1.283 * (np.power(pred_thFC_60, 2))) - (0.374 * pred_thFC_60) - 0.015
                    #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                    ts_60 =soils_df_pivot["theta_s"][i]
                    ks_60=(soils_df_pivot['ksat'][i])*240
                if row['depth'] == "5_15":
                    pred_thWP_15 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                    wp_15 = pred_thWP_15 + (0.14 * pred_thWP_15) - 0.02
                    pred_thFC_15 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                    fc_15 = pred_thFC_15 + (1.283 * (np.power(pred_thFC_15, 2))) - (0.374 * pred_thFC_15) - 0.015
                    #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                    ts_15 =soils_df_pivot["theta_s"][i]
                    #print(ts_15)
                    ks_15=(soils_df_pivot['ksat'][i])*240
                if row['depth'] == "60_100":
                    pred_thWP_100 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                    wp_100 = pred_thWP_100 + (0.14 * pred_thWP_100) - 0.02
                    pred_thFC_100 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                    fc_100 = pred_thFC_100 + (1.283 * (np.power(pred_thFC_100, 2))) - (0.374 * pred_thFC_100) - 0.015
                    #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                    ts_100 =soils_df_pivot["theta_s"][i]
                    ks_100=(soils_df_pivot['ksat'][i])*240



                    custom = Soil('custom',cn=46,rew=7, dz=[0.025]*2+[0.05]*2+[0.075]*2+[0.15]*2+[0.2]*2+[0.5]*2)

                    custom.add_layer(thickness=0.05,thS=ts_5, # assuming soil properties are the same in the upper 0.1m
                         Ksat=ks_5,thWP =wp_5 , 
                         thFC = fc_5, penetrability = 100.0)
                    custom.add_layer(thickness=0.15,thS=ts_15, # assuming soil properties are the same in the upper 0.1m
                                 Ksat=ks_15,thWP =wp_15 , 
                                 thFC = fc_15, penetrability = 100.0)
                    custom.add_layer(thickness=0.3,thS=ts_30, # assuming soil properties are the same in the upper 0.1m
                                 Ksat=ks_30,thWP =wp_30 , 
                                 thFC = fc_30, penetrability = 100.0)
                    custom.add_layer(thickness=0.3,thS=ts_60, # assuming soil properties are the same in the upper 0.1m
                                 Ksat=ks_60,thWP =wp_60 , 
                                 thFC = fc_60, penetrability = 100.0)
                    custom.add_layer(thickness=0.4,thS=ts_100, # assuming soil properties are the same in the upper 0.1m
                                 Ksat=ks_100,thWP =wp_100 , 
                                 thFC = fc_100, penetrability = 100.0)
                    custom.add_layer(thickness=1,thS=ts_200, # assuming soil properties are the same in the upper 0.1m
                                 Ksat=ks_200,thWP =wp_200 , 
                                 thFC = fc_200, penetrability = 100.0)
                    custom_soil.append(custom)
                    #df_list.append(custom)

                    #print(custom)

    return(custom_soil)

In [4]:
with open(wd + '/eggs/data/input_dict.pickle', 'rb') as input_data: 
    input_dict = pickle.load(input_data) 

In [ ]:
input_dict.keys()

In [5]:
irrig_corn = '1'
irrig_corn_dict = {k:v for (k,v) in input_dict.items() if irrig_corn in k}

In [ ]:
# calibrate using the decatur, gove, graham, logan, rawlins
calibration = list(irrig_corn_dict.items())[:5]
calibration

In [ ]:
calibration[0][1]

In [13]:
# getting the different datasets
gridmet = [sublist[1][0] for sublist in calibration] # data stored in nested list with 0 having the 
lai = [sublist[1][1] for sublist in calibration]
soil = [sublist[1][2] for sublist in calibration]

In [34]:
soil[0]

var,crop_mn_codeyear,Year,depth,alpha,clay,hb,ksat,lambda,n,om,sand,silt,theta_r,theta_s
0,1_Decatur,2006.0,0_5,-0.415029,21.677830,0.416280,1.582628,0.298007,1.313759,1.561598,16.906940,58.423781,0.058815,0.499963
1,1_Decatur,2006.0,100_200,-0.417431,17.985571,0.416632,1.747216,0.315498,1.336323,0.174695,19.357758,59.331870,0.054213,0.488484
2,1_Decatur,2006.0,15_30,-0.428745,25.289850,0.435674,1.334489,0.285533,1.295082,1.047764,15.133924,55.992885,0.064739,0.501657
3,1_Decatur,2006.0,30_60,-0.451755,25.984820,0.454456,1.089989,0.284164,1.293146,0.607766,15.122233,55.539237,0.068000,0.499710
4,1_Decatur,2006.0,5_15,-0.422424,22.549341,0.426578,1.470932,0.294614,1.308546,1.423417,16.368427,57.726577,0.060534,0.495986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,1_Decatur,2020.0,100_200,-0.417431,17.985571,0.416632,1.747216,0.315498,1.336323,0.174695,19.357758,59.331870,0.054213,0.488484
86,1_Decatur,2020.0,15_30,-0.428745,25.289850,0.435674,1.334489,0.285533,1.295082,1.047764,15.133924,55.992885,0.064739,0.501657
87,1_Decatur,2020.0,30_60,-0.451755,25.984820,0.454456,1.089989,0.284164,1.293146,0.607766,15.122233,55.539237,0.068000,0.499710
88,1_Decatur,2020.0,5_15,-0.422424,22.549341,0.426578,1.470932,0.294614,1.308546,1.423417,16.368427,57.726577,0.060534,0.495986


### Soil data is currently being saved in one list - wondering if saving it in 5 lists make more sense (one list for each county)

In [75]:
custom_soil = []
custom_soil2 = []
for df in soil:
    
    #soils_df = df.groupby(['Year'])
    soils_df = [v for k, v in df.groupby('Year')]
    #print(soils_df)
    for j in soils_df:
        #df_list = []
        soils_df_pivot = j.drop(['crop_mn_codeyear', 'Year'], axis=1)
        #print(soils_df_pivot)
        for i, row in soils_df_pivot.iterrows():
            #print(row)
            #print(soils_df_pivot)
            if row['depth'] == "0_5":
                pred_thWP_5 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                wp_5 = pred_thWP_5 + (0.14 * pred_thWP_5) - 0.02
                pred_thFC_5 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                fc_5 = pred_thFC_5 + (1.283 * (np.power(pred_thFC_5, 2))) - (0.374 * pred_thFC_5) - 0.015
                    #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                ts_5 =soils_df_pivot["theta_s"][i]
                ks_5=(soils_df_pivot['ksat'][i])*240
            if row['depth'] == "100_200":
                pred_thWP_200 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                wp_200 = pred_thWP_200 + (0.14 * pred_thWP_200) - 0.02
                pred_thFC_200 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                fc_200 = pred_thFC_200 + (1.283 * (np.power(pred_thFC_200, 2))) - (0.374 * pred_thFC_200) - 0.015
                #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                ts_200 =soils_df_pivot["theta_s"][i]
                ks_200=(soils_df_pivot['ksat'][i])*240
            if row['depth'] == "15_30":
                pred_thWP_30 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                wp_30 = pred_thWP_30 + (0.14 * pred_thWP_30) - 0.02
                pred_thFC_30 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                fc_30 = pred_thFC_30 + (1.283 * (np.power(pred_thFC_30, 2))) - (0.374 * pred_thFC_30) - 0.015
                #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                ts_30 =soils_df_pivot["theta_s"][i]
                ks_30=(soils_df_pivot['ksat'][i])*240
            if row['depth'] == "30_60":
                pred_thWP_60 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                wp_60 = pred_thWP_60 + (0.14 * pred_thWP_60) - 0.02
                pred_thFC_60 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                fc_60 = pred_thFC_60 + (1.283 * (np.power(pred_thFC_60, 2))) - (0.374 * pred_thFC_60) - 0.015
                #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                ts_60 =soils_df_pivot["theta_s"][i]
                ks_60=(soils_df_pivot['ksat'][i])*240
            if row['depth'] == "5_15":
                pred_thWP_15 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                wp_15 = pred_thWP_15 + (0.14 * pred_thWP_15) - 0.02
                pred_thFC_15 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                fc_15 = pred_thFC_15 + (1.283 * (np.power(pred_thFC_15, 2))) - (0.374 * pred_thFC_15) - 0.015
                #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                ts_15 =soils_df_pivot["theta_s"][i]
                #print(ts_15)
                ks_15=(soils_df_pivot['ksat'][i])*240
            if row['depth'] == "60_100":
                pred_thWP_100 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                wp_100 = pred_thWP_100 + (0.14 * pred_thWP_100) - 0.02
                pred_thFC_100 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                fc_100 = pred_thFC_100 + (1.283 * (np.power(pred_thFC_100, 2))) - (0.374 * pred_thFC_100) - 0.015
                #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                ts_100 =soils_df_pivot["theta_s"][i]
                ks_100=(soils_df_pivot['ksat'][i])*240
            
                
                
                custom = Soil('custom',cn=46,rew=7, dz=[0.025]*2+[0.05]*2+[0.075]*2+[0.15]*2+[0.2]*2+[0.5]*2)

                custom.add_layer(thickness=0.05,thS=ts_5, # assuming soil properties are the same in the upper 0.1m
                     Ksat=ks_5,thWP =wp_5 , 
                     thFC = fc_5, penetrability = 100.0)
                custom.add_layer(thickness=0.15,thS=ts_15, # assuming soil properties are the same in the upper 0.1m
                             Ksat=ks_15,thWP =wp_15 , 
                             thFC = fc_15, penetrability = 100.0)
                custom.add_layer(thickness=0.3,thS=ts_30, # assuming soil properties are the same in the upper 0.1m
                             Ksat=ks_30,thWP =wp_30 , 
                             thFC = fc_30, penetrability = 100.0)
                custom.add_layer(thickness=0.3,thS=ts_60, # assuming soil properties are the same in the upper 0.1m
                             Ksat=ks_60,thWP =wp_60 , 
                             thFC = fc_60, penetrability = 100.0)
                custom.add_layer(thickness=0.4,thS=ts_100, # assuming soil properties are the same in the upper 0.1m
                             Ksat=ks_100,thWP =wp_100 , 
                             thFC = fc_100, penetrability = 100.0)
                custom.add_layer(thickness=1,thS=ts_200, # assuming soil properties are the same in the upper 0.1m
                             Ksat=ks_200,thWP =wp_200 , 
                             thFC = fc_200, penetrability = 100.0)
                custom_soil.append(custom)
                #df_list.append(custom)
    
                #print(custom)


                

  

In [81]:
soilfg = soilCompart(soil)
soilfg

Name: custom
zSoil: 2.0
nComp: 12
nLayer: 6
adj_rew: 1
rew: 7
calc_cn: 0
cn: 46
z_res: -999
evap_z_surf: 0.04
evap_z_min: 0.15
evap_z_max: 0.3
kex: 1.1
f_evap: 4
f_wrel_exp: 0.4
fwcc: 50
z_cn: 0.3
z_germ: 0.3
adj_cn: 1
fshape_cr: 16
z_top: 0.1
Name: custom
zSoil: 2.0
nComp: 12
nLayer: 6
adj_rew: 1
rew: 7
calc_cn: 0
cn: 46
z_res: -999
evap_z_surf: 0.04
evap_z_min: 0.15
evap_z_max: 0.3
kex: 1.1
f_evap: 4
f_wrel_exp: 0.4
fwcc: 50
z_cn: 0.3
z_germ: 0.3
adj_cn: 1
fshape_cr: 16
z_top: 0.1
Name: custom
zSoil: 2.0
nComp: 12
nLayer: 6
adj_rew: 1
rew: 7
calc_cn: 0
cn: 46
z_res: -999
evap_z_surf: 0.04
evap_z_min: 0.15
evap_z_max: 0.3
kex: 1.1
f_evap: 4
f_wrel_exp: 0.4
fwcc: 50
z_cn: 0.3
z_germ: 0.3
adj_cn: 1
fshape_cr: 16
z_top: 0.1
Name: custom
zSoil: 2.0
nComp: 12
nLayer: 6
adj_rew: 1
rew: 7
calc_cn: 0
cn: 46
z_res: -999
evap_z_surf: 0.04
evap_z_min: 0.15
evap_z_max: 0.3
kex: 1.1
f_evap: 4
f_wrel_exp: 0.4
fwcc: 50
z_cn: 0.3
z_germ: 0.3
adj_cn: 1
fshape_cr: 16
z_top: 0.1
Name: custom
zSoil: 2.0


[ ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ,
  ]